In [7]:
import numpy as np 
import pandas as pd

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

#import scispacy
#import spacy

#from scipy.spatial.distance import jensenshannon

#import joblib

#import pyLDAvis
#import pyLDAvis.sklearn
#pyLDAvis.enable_notebook()

from IPython.display import HTML, display

from ipywidgets import interact, Layout, HBox, VBox, Box
import ipywidgets as widgets
from IPython.display import clear_output

#from tqdm import tqdm

In [8]:
df = pd.read_csv('Data/paper.csv')

In [34]:
df.head()

,Unnamed: 0,abstract,doc_id,source,text_body,title
0,0,NaN,NaN,NaN,,NaN
1,1,word count: 194 22 Text word count: 5168 23 24...,0015023cc06b5362d332b3baf348d11567ca2fbb,biorxiv_medrxiv,"VP3, and VP0 (which is further processed to VP...",The RNA pseudoknots in foot-and-mouth disease ...
2,2,NaN,004f0f8bb66cf446678dc13cf2701feec4f36d76,biorxiv_medrxiv,The 2019-nCoV epidemic has spread across China...,Healthcare-resource-adjusted vulnerabilities t...
3,3,Infectious bronchitis (IB) causes significant ...,00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,biorxiv_medrxiv,"Infectious bronchitis (IB), which is caused by...","Real-time, MinION-based, amplicon sequencing f..."
4,4,Nipah Virus (NiV) came into limelight recently...,0139ea4ca580af99b602c6435368e7fdbefacb03,biorxiv_medrxiv,Nipah is an infectious negative-sense single-s...,A Combined Evidence Approach to Prioritize Nip...


In [21]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

ImportError: cannot import name 'Stemmer'

In [18]:
df[['text_body']] = df[['text_body']].fillna('')
data_text = df[['text_body']]
data_text['index'] = data_text.index
documents = data_text

C:\Data\Python\anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
documents

,text_body,index
0,,0
1,"VP3, and VP0 (which is further processed to VP...",1
2,The 2019-nCoV epidemic has spread across China...,2
3,"Infectious bronchitis (IB), which is caused by...",3
4,Nipah is an infectious negative-sense single-s...,4
5,"In December 2019, a cluster of patients with p...",5
6,The sudden outbreak of the new coronavirus (SA...,6
7,The copyright holder for this preprint (which ...,7
8,they affect seasonal influenza dynamics. The m...,8
9,"siRNA without relying on plant transformation,...",9


In [13]:
print(len(documents))
print(documents[:5])

29316
                                           text_body  index
0                                                NaN      0
1  VP3, and VP0 (which is further processed to VP...      1
2  The 2019-nCoV epidemic has spread across China...      2
3  Infectious bronchitis (IB), which is caused by...      3
4  Nipah is an infectious negative-sense single-s...      4


In [27]:
stemmer = SnowballStemmer("english")

In [28]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [29]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['Hemophagocytic', 'lymphohistiocytosis', '(HLH)', 'is', 'a', 'syndrome', 'that', 'is', 'characterized', 'by', 'an', 'inappropriate', 'hyper-inflammatory', 'immune', 'responseprimary,', 'as', 'a', 'consequence', 'of', 'a', 'genetic', 'defect', 'of', 'NK', 'cells', 'and', 'cytotoxic', 'T', 'lymphocytes', 'orsecondary,', 'in', 'the', 'progression', 'of', 'infections,', 'rheumatic', 'or', 'autoimmune', 'diseases,', 'malignancies', 'or', 'metabolic', 'diseases.', 'Among', 'HLH-related', 'infections,', 'the', 'most', 'common', 'are', 'the', 'viral', 'infections:', 'Epstein--Barr', 'virus,', 'cytomegalovirus,', 'other', 'herpes', 'viruses,', 'the', 'viruses', 'of', 'hepatitis', 'B', 'and', 'C,', 'etc.,', 'followed', 'by', 'bacterial,', 'parasitic,', 'or', 'fungal', 'infections.', 'Association', 'of', 'secondary', 'HLH', 'to', 'Streptococcus', 'pneumoniae', 'infection', 'in', 'a', 'patient', 'splenectomized', 'for', 'spherocytosis', 'has', 'not', 'been', 'described', 'so',

In [30]:
processed_docs = documents['text_body'].map(preprocess)
processed_docs[:10]

0                                                   []
1    [process, virus, assembl, region, encod, struc...
2    [ncov, epidem, spread, china, countri, februar...
3    [infecti, bronchiti, caus, infecti, bronchiti,...
4    [nipah, infecti, negat, sens, singl, strand, v...
5    [decemb, cluster, patient, pneumonia, unknown,...
6    [sudden, outbreak, coronavirus, sar, decemb, p...
7    [copyright, holder, preprint, peer, review, au...
8    [affect, season, influenza, dynam, model, appl...
9    [sirna, reli, plant, transform, virus, limit, ...
Name: text_body, dtype: object

In [31]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 aatii
1 abl
2 absenc
3 achiev
4 addit
5 advantag
6 advers
7 aflii
8 agre
9 algorithm
10 alighn


In [32]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [33]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(2, 1),
 (31, 1),
 (37, 1),
 (48, 1),
 (92, 1),
 (100, 1),
 (152, 1),
 (167, 1),
 (179, 1),
 (242, 1),
 (257, 5),
 (265, 1),
 (269, 2),
 (305, 1),
 (307, 1),
 (312, 5),
 (313, 1),
 (319, 1),
 (356, 3),
 (365, 1),
 (375, 2),
 (392, 1),
 (409, 1),
 (424, 1),
 (429, 1),
 (453, 1),
 (473, 1),
 (474, 2),
 (477, 1),
 (478, 1),
 (495, 1),
 (503, 1),
 (520, 1),
 (545, 4),
 (548, 1),
 (556, 8),
 (558, 1),
 (565, 1),
 (593, 3),
 (619, 1),
 (656, 1),
 (668, 2),
 (688, 1),
 (702, 1),
 (721, 3),
 (731, 1),
 (751, 1),
 (752, 3),
 (756, 1),
 (767, 1),
 (852, 1),
 (870, 1),
 (875, 1),
 (877, 1),
 (884, 1),
 (894, 1),
 (944, 2),
 (958, 1),
 (967, 4),
 (1011, 1),
 (1050, 9),
 (1055, 2),
 (1099, 1),
 (1125, 2),
 (1135, 1),
 (1140, 1),
 (1147, 2),
 (1148, 1),
 (1160, 1),
 (1171, 1),
 (1174, 1),
 (1177, 2),
 (1189, 4),
 (1220, 2),
 (1226, 2),
 (1243, 1),
 (1245, 2),
 (1253, 1),
 (1277, 1),
 (1309, 5),
 (1314, 1),
 (1406, 4),
 (1409, 1),
 (1457, 1),
 (1491, 2),
 (1545, 1),
 (1576, 1),
 (1615, 1),
 (1629, 1

In [ ]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
processed_docs[4310]

In [ ]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

In [ ]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

In [ ]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))